# Jaringan Saraf Rekuren

Dalam modul sebelumnya, kita telah menggunakan representasi semantik yang kaya dari teks, dan sebuah pengklasifikasi linear sederhana di atas embedding. Arsitektur ini menangkap makna agregat dari kata-kata dalam sebuah kalimat, tetapi tidak memperhitungkan **urutan** kata-kata, karena operasi agregasi pada embedding menghilangkan informasi ini dari teks asli. Karena model-model ini tidak mampu memodelkan urutan kata, mereka tidak dapat menyelesaikan tugas yang lebih kompleks atau ambigu seperti pembuatan teks atau menjawab pertanyaan.

Untuk menangkap makna dari urutan teks, kita perlu menggunakan arsitektur jaringan saraf lain, yang disebut **jaringan saraf rekuren**, atau RNN. Dalam RNN, kita melewatkan kalimat kita melalui jaringan satu simbol pada satu waktu, dan jaringan menghasilkan beberapa **state**, yang kemudian kita teruskan kembali ke jaringan bersama simbol berikutnya.

Diberikan urutan token $X_0,\dots,X_n$, RNN menciptakan urutan blok jaringan saraf, dan melatih urutan ini secara end-to-end menggunakan back propagation. Setiap blok jaringan mengambil pasangan $(X_i,S_i)$ sebagai input, dan menghasilkan $S_{i+1}$ sebagai hasil. State akhir $S_n$ atau output $X_n$ masuk ke pengklasifikasi linear untuk menghasilkan hasil akhir. Semua blok jaringan berbagi bobot yang sama, dan dilatih secara end-to-end menggunakan satu kali proses back propagation.

Karena vektor state $S_0,\dots,S_n$ diteruskan melalui jaringan, RNN mampu mempelajari ketergantungan berurutan antar kata. Sebagai contoh, ketika kata *tidak* muncul di suatu tempat dalam urutan, RNN dapat belajar untuk menegasikan elemen tertentu dalam vektor state, yang menghasilkan negasi.

> Karena bobot semua blok RNN pada gambar dibagikan, gambar yang sama dapat direpresentasikan sebagai satu blok (di sebelah kanan) dengan loop umpan balik rekuren, yang meneruskan state output jaringan kembali ke input.

Mari kita lihat bagaimana jaringan saraf rekuren dapat membantu kita mengklasifikasikan dataset berita kita.


In [1]:
import torch
import torchtext
from torchnlp import *
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_size = len(vocab)

Loading dataset...
Building vocab...


## Klasifikasi RNN Sederhana

Dalam kasus RNN sederhana, setiap unit rekuren adalah jaringan linear sederhana, yang menerima vektor input yang digabungkan dengan vektor status, dan menghasilkan vektor status baru. PyTorch merepresentasikan unit ini dengan kelas `RNNCell`, dan jaringan dari unit-unit tersebut sebagai lapisan `RNN`.

Untuk mendefinisikan sebuah klasifikasi RNN, kita akan terlebih dahulu menerapkan lapisan embedding untuk menurunkan dimensi kosakata input, lalu menambahkan lapisan RNN di atasnya:


In [2]:
class RNNClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.rnn = torch.nn.RNN(embed_dim,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x):
        batch_size = x.size(0)
        x = self.embedding(x)
        x,h = self.rnn(x)
        return self.fc(x.mean(dim=1))

> **Catatan:** Di sini kita menggunakan lapisan embedding yang belum dilatih untuk kesederhanaan, tetapi untuk hasil yang lebih baik kita dapat menggunakan lapisan embedding yang telah dilatih sebelumnya dengan Word2Vec atau GloVe embeddings, seperti yang dijelaskan di unit sebelumnya. Untuk pemahaman yang lebih baik, Anda mungkin ingin menyesuaikan kode ini agar bekerja dengan embedding yang telah dilatih sebelumnya.

Dalam kasus kita, kita akan menggunakan data loader yang telah dipad, sehingga setiap batch akan memiliki sejumlah urutan yang dipad dengan panjang yang sama. Lapisan RNN akan mengambil urutan tensor embedding, dan menghasilkan dua output: 
* $x$ adalah urutan output sel RNN di setiap langkah
* $h$ adalah keadaan tersembunyi akhir untuk elemen terakhir dari urutan

Kemudian kita menerapkan pengklasifikasi linier yang sepenuhnya terhubung untuk mendapatkan jumlah kelas.

> **Catatan:** RNN cukup sulit untuk dilatih, karena setelah sel RNN diurai sepanjang panjang urutan, jumlah lapisan yang terlibat dalam propagasi balik menjadi cukup besar. Oleh karena itu, kita perlu memilih tingkat pembelajaran yang kecil, dan melatih jaringan pada dataset yang lebih besar untuk menghasilkan hasil yang baik. Proses ini bisa memakan waktu cukup lama, sehingga penggunaan GPU lebih disarankan.


In [3]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=padify, shuffle=True)
net = RNNClassifier(vocab_size,64,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=0.001)

3200: acc=0.3090625
6400: acc=0.38921875
9600: acc=0.4590625
12800: acc=0.511953125
16000: acc=0.5506875
19200: acc=0.57921875
22400: acc=0.6070089285714285
25600: acc=0.6304296875
28800: acc=0.6484027777777778
32000: acc=0.66509375
35200: acc=0.6790056818181818
38400: acc=0.6929166666666666
41600: acc=0.7035817307692308
44800: acc=0.7137276785714286
48000: acc=0.72225
51200: acc=0.73001953125
54400: acc=0.7372794117647059
57600: acc=0.7436631944444444
60800: acc=0.7503947368421052
64000: acc=0.75634375
67200: acc=0.7615773809523809
70400: acc=0.7662642045454545
73600: acc=0.7708423913043478
76800: acc=0.7751822916666666
80000: acc=0.7790625
83200: acc=0.7825
86400: acc=0.7858564814814815
89600: acc=0.7890513392857142
92800: acc=0.7920474137931034
96000: acc=0.7952708333333334
99200: acc=0.7982258064516129
102400: acc=0.80099609375
105600: acc=0.8037594696969697
108800: acc=0.8060569852941176


## Long Short Term Memory (LSTM)

Salah satu masalah utama pada RNN klasik adalah masalah yang disebut **vanishing gradients**. Karena RNN dilatih secara end-to-end dalam satu proses back-propagation, sulit untuk menyebarkan error ke lapisan pertama jaringan, sehingga jaringan tidak dapat mempelajari hubungan antara token yang berjauhan. Salah satu cara untuk menghindari masalah ini adalah dengan memperkenalkan **manajemen keadaan secara eksplisit** menggunakan apa yang disebut **gates**. Ada dua arsitektur paling terkenal dari jenis ini: **Long Short Term Memory** (LSTM) dan **Gated Relay Unit** (GRU).

![Gambar menunjukkan contoh sel long short term memory](../../../../../lessons/5-NLP/16-RNN/images/long-short-term-memory-cell.svg)

Jaringan LSTM diorganisasikan dengan cara yang mirip dengan RNN, tetapi ada dua keadaan yang diteruskan dari lapisan ke lapisan: keadaan aktual $c$, dan vektor tersembunyi $h$. Pada setiap unit, vektor tersembunyi $h_i$ digabungkan dengan input $x_i$, dan mereka mengontrol apa yang terjadi pada keadaan $c$ melalui **gates**. Setiap gate adalah jaringan neural dengan aktivasi sigmoid (output dalam rentang $[0,1]$), yang dapat dianggap sebagai masker bitwise ketika dikalikan dengan vektor keadaan. Berikut adalah gates yang ada (dari kiri ke kanan pada gambar di atas):
* **forget gate** mengambil vektor tersembunyi dan menentukan komponen mana dari vektor $c$ yang perlu dilupakan, dan mana yang perlu diteruskan.
* **input gate** mengambil beberapa informasi dari input dan vektor tersembunyi, lalu memasukkannya ke dalam keadaan.
* **output gate** mengubah keadaan melalui beberapa lapisan linear dengan aktivasi $\tanh$, kemudian memilih beberapa komponennya menggunakan vektor tersembunyi $h_i$ untuk menghasilkan keadaan baru $c_{i+1}$.

Komponen dari keadaan $c$ dapat dianggap sebagai beberapa flag yang dapat diaktifkan atau dinonaktifkan. Sebagai contoh, ketika kita menemukan nama *Alice* dalam urutan, kita mungkin ingin mengasumsikan bahwa itu merujuk pada karakter perempuan, dan mengaktifkan flag dalam keadaan bahwa kita memiliki kata benda perempuan dalam kalimat. Ketika kita kemudian menemukan frasa *and Tom*, kita akan mengaktifkan flag bahwa kita memiliki kata benda jamak. Dengan demikian, dengan memanipulasi keadaan, kita dapat secara teoritis melacak sifat gramatikal dari bagian-bagian kalimat.

> **Note**: Sumber yang sangat bagus untuk memahami struktur internal LSTM adalah artikel hebat ini [Understanding LSTM Networks](https://colah.github.io/posts/2015-08-Understanding-LSTMs/) oleh Christopher Olah.

Meskipun struktur internal sel LSTM mungkin terlihat kompleks, PyTorch menyembunyikan implementasi ini di dalam kelas `LSTMCell`, dan menyediakan objek `LSTM` untuk merepresentasikan seluruh lapisan LSTM. Dengan demikian, implementasi classifier LSTM akan sangat mirip dengan RNN sederhana yang telah kita lihat sebelumnya:


In [4]:
class LSTMClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.embedding.weight.data = torch.randn_like(self.embedding.weight.data)-0.5
        self.rnn = torch.nn.LSTM(embed_dim,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x):
        batch_size = x.size(0)
        x = self.embedding(x)
        x,(h,c) = self.rnn(x)
        return self.fc(h[-1])

In [5]:
net = LSTMClassifier(vocab_size,64,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=0.001)

3200: acc=0.259375
6400: acc=0.25859375
9600: acc=0.26177083333333334
12800: acc=0.2784375
16000: acc=0.313
19200: acc=0.3528645833333333
22400: acc=0.3965625
25600: acc=0.4385546875
28800: acc=0.4752777777777778
32000: acc=0.505375
35200: acc=0.5326704545454546
38400: acc=0.5557552083333334
41600: acc=0.5760817307692307
44800: acc=0.5954910714285714
48000: acc=0.6118333333333333
51200: acc=0.62681640625
54400: acc=0.6404779411764706
57600: acc=0.6520138888888889
60800: acc=0.662828947368421
64000: acc=0.673546875
67200: acc=0.6831547619047619
70400: acc=0.6917897727272727
73600: acc=0.6997146739130434
76800: acc=0.707109375
80000: acc=0.714075
83200: acc=0.7209134615384616
86400: acc=0.727037037037037
89600: acc=0.7326674107142858
92800: acc=0.7379633620689655
96000: acc=0.7433645833333333
99200: acc=0.7479032258064516
102400: acc=0.752119140625
105600: acc=0.7562405303030303
108800: acc=0.76015625
112000: acc=0.7641339285714286
115200: acc=0.7677777777777778
118400: acc=0.77112331081

(0.03487814127604167, 0.7728)

## Urutan yang Dikemas

Dalam contoh kita, kita harus mengisi semua urutan dalam minibatch dengan vektor nol. Meskipun ini mengakibatkan pemborosan memori, dengan RNN lebih kritis bahwa sel-sel RNN tambahan dibuat untuk item input yang diisi, yang ikut serta dalam pelatihan, tetapi tidak membawa informasi input yang penting. Akan jauh lebih baik jika melatih RNN hanya pada ukuran urutan yang sebenarnya.

Untuk melakukan itu, format khusus penyimpanan urutan yang diisi diperkenalkan di PyTorch. Misalkan kita memiliki minibatch input yang diisi yang terlihat seperti ini:
```
[[1,2,3,4,5],
 [6,7,8,0,0],
 [9,0,0,0,0]]
```
Di sini 0 mewakili nilai yang diisi, dan vektor panjang sebenarnya dari urutan input adalah `[5,3,1]`.

Untuk melatih RNN secara efektif dengan urutan yang diisi, kita ingin memulai pelatihan grup pertama sel RNN dengan minibatch besar (`[1,6,9]`), tetapi kemudian mengakhiri pemrosesan urutan ketiga, dan melanjutkan pelatihan dengan minibatch yang lebih pendek (`[2,7]`, `[3,8]`), dan seterusnya. Dengan demikian, urutan yang dikemas direpresentasikan sebagai satu vektor - dalam kasus kita `[1,6,9,2,7,3,8,4,5]`, dan vektor panjang (`[5,3,1]`), dari mana kita dapat dengan mudah merekonstruksi minibatch yang diisi asli.

Untuk menghasilkan urutan yang dikemas, kita dapat menggunakan fungsi `torch.nn.utils.rnn.pack_padded_sequence`. Semua lapisan berulang, termasuk RNN, LSTM, dan GRU, mendukung urutan yang dikemas sebagai input, dan menghasilkan output yang dikemas, yang dapat didekode menggunakan `torch.nn.utils.rnn.pad_packed_sequence`.

Untuk dapat menghasilkan urutan yang dikemas, kita perlu memberikan vektor panjang ke jaringan, dan dengan demikian kita memerlukan fungsi yang berbeda untuk mempersiapkan minibatch:


In [6]:
def pad_length(b):
    # build vectorized sequence
    v = [encode(x[1]) for x in b]
    # compute max length of a sequence in this minibatch and length sequence itself
    len_seq = list(map(len,v))
    l = max(len_seq)
    return ( # tuple of three tensors - labels, padded features, length sequence
        torch.LongTensor([t[0]-1 for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v]),
        torch.tensor(len_seq)
    )

train_loader_len = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=pad_length, shuffle=True)

Jaringan aktual akan sangat mirip dengan `LSTMClassifier` di atas, tetapi proses `forward` akan menerima baik minibatch yang telah dipadatkan maupun vektor panjang urutan. Setelah menghitung embedding, kita menghitung packed sequence, meneruskannya ke lapisan LSTM, dan kemudian membongkar hasilnya kembali.

> **Catatan**: Sebenarnya kita tidak menggunakan hasil yang telah dibongkar `x`, karena kita menggunakan output dari lapisan tersembunyi dalam perhitungan berikutnya. Oleh karena itu, kita dapat menghapus proses pembongkaran sepenuhnya dari kode ini. Alasan kita meletakkannya di sini adalah agar Anda dapat dengan mudah memodifikasi kode ini, jika Anda perlu menggunakan output jaringan dalam perhitungan lebih lanjut.


In [7]:
class LSTMPackClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.embedding.weight.data = torch.randn_like(self.embedding.weight.data)-0.5
        self.rnn = torch.nn.LSTM(embed_dim,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x, lengths):
        batch_size = x.size(0)
        x = self.embedding(x)
        pad_x = torch.nn.utils.rnn.pack_padded_sequence(x,lengths,batch_first=True,enforce_sorted=False)
        pad_x,(h,c) = self.rnn(pad_x)
        x, _ = torch.nn.utils.rnn.pad_packed_sequence(pad_x,batch_first=True)
        return self.fc(h[-1])

In [8]:
net = LSTMPackClassifier(vocab_size,64,32,len(classes)).to(device)
train_epoch_emb(net,train_loader_len, lr=0.001,use_pack_sequence=True)


3200: acc=0.285625
6400: acc=0.33359375
9600: acc=0.3876041666666667
12800: acc=0.44078125
16000: acc=0.4825
19200: acc=0.5235416666666667
22400: acc=0.5559821428571429
25600: acc=0.58609375
28800: acc=0.6116666666666667
32000: acc=0.63340625
35200: acc=0.6525284090909091
38400: acc=0.668515625
41600: acc=0.6822596153846154
44800: acc=0.6948214285714286
48000: acc=0.7052708333333333
51200: acc=0.71521484375
54400: acc=0.7239889705882353
57600: acc=0.7315277777777778
60800: acc=0.7388486842105263
64000: acc=0.74571875
67200: acc=0.7518303571428572
70400: acc=0.7576988636363636
73600: acc=0.7628940217391305
76800: acc=0.7681510416666667
80000: acc=0.7728125
83200: acc=0.7772235576923077
86400: acc=0.7815393518518519
89600: acc=0.7857700892857142
92800: acc=0.7895043103448276
96000: acc=0.7930520833333333
99200: acc=0.7959072580645161
102400: acc=0.798994140625
105600: acc=0.802064393939394
108800: acc=0.8051378676470589
112000: acc=0.8077857142857143
115200: acc=0.8104600694444445
118400

(0.029785829671223958, 0.8138166666666666)

> **Catatan:** Anda mungkin telah memperhatikan parameter `use_pack_sequence` yang kami berikan ke fungsi pelatihan. Saat ini, fungsi `pack_padded_sequence` membutuhkan tensor urutan panjang berada di perangkat CPU, dan oleh karena itu fungsi pelatihan perlu menghindari memindahkan data urutan panjang ke GPU saat pelatihan. Anda dapat melihat implementasi fungsi `train_emb` dalam file [`torchnlp.py`](../../../../../lessons/5-NLP/16-RNN/torchnlp.py).


## RNN Bidirectional dan Multilayer

Dalam contoh-contoh kita, semua jaringan rekuren bekerja dalam satu arah, dari awal hingga akhir sebuah urutan. Hal ini terlihat alami, karena menyerupai cara kita membaca dan mendengarkan percakapan. Namun, karena dalam banyak kasus praktis kita memiliki akses acak ke urutan input, masuk akal untuk menjalankan komputasi rekuren dalam kedua arah. Jaringan seperti ini disebut **RNN bidirectional**, dan dapat dibuat dengan menambahkan parameter `bidirectional=True` pada konstruktor RNN/LSTM/GRU.

Saat bekerja dengan jaringan bidirectional, kita memerlukan dua vektor state tersembunyi, satu untuk setiap arah. PyTorch mengkodekan vektor-vektor tersebut sebagai satu vektor dengan ukuran dua kali lebih besar, yang cukup praktis, karena biasanya Anda akan meneruskan state tersembunyi yang dihasilkan ke lapisan linear fully-connected, dan Anda hanya perlu memperhitungkan peningkatan ukuran ini saat membuat lapisan tersebut.

Jaringan rekuren, baik satu arah maupun bidirectional, menangkap pola tertentu dalam sebuah urutan, dan dapat menyimpannya ke dalam vektor state atau meneruskannya ke output. Seperti pada jaringan konvolusional, kita dapat membangun lapisan rekuren lain di atas lapisan pertama untuk menangkap pola tingkat yang lebih tinggi, yang dibangun dari pola tingkat rendah yang diekstraksi oleh lapisan pertama. Ini membawa kita pada konsep **RNN multilayer**, yang terdiri dari dua atau lebih jaringan rekuren, di mana output dari lapisan sebelumnya diteruskan ke lapisan berikutnya sebagai input.

![Gambar yang menunjukkan Multilayer long-short-term-memory- RNN](../../../../../translated_images/id/multi-layer-lstm.dd975e29bb2a59fe.webp)

*Gambar dari [postingan luar biasa ini](https://towardsdatascience.com/from-a-lstm-cell-to-a-multilayer-lstm-network-with-pytorch-2899eb5696f3) oleh Fernando López*

PyTorch mempermudah pembuatan jaringan seperti ini, karena Anda hanya perlu menambahkan parameter `num_layers` pada konstruktor RNN/LSTM/GRU untuk secara otomatis membangun beberapa lapisan rekuren. Hal ini juga berarti bahwa ukuran vektor state/tersembunyi akan meningkat secara proporsional, dan Anda perlu memperhitungkan hal ini saat menangani output dari lapisan rekuren.


## RNN untuk tugas lainnya

Dalam unit ini, kita telah melihat bahwa RNN dapat digunakan untuk klasifikasi urutan, tetapi sebenarnya, RNN dapat menangani banyak tugas lainnya, seperti pembuatan teks, penerjemahan mesin, dan lainnya. Kita akan membahas tugas-tugas tersebut di unit berikutnya.



---

**Penafian**:  
Dokumen ini telah diterjemahkan menggunakan layanan penerjemahan AI [Co-op Translator](https://github.com/Azure/co-op-translator). Meskipun kami berusaha untuk memberikan hasil yang akurat, harap diingat bahwa terjemahan otomatis mungkin mengandung kesalahan atau ketidakakuratan. Dokumen asli dalam bahasa aslinya harus dianggap sebagai sumber yang otoritatif. Untuk informasi yang bersifat kritis, disarankan menggunakan jasa penerjemahan profesional oleh manusia. Kami tidak bertanggung jawab atas kesalahpahaman atau penafsiran yang keliru yang timbul dari penggunaan terjemahan ini.
